In [1]:
%load_ext autoreload
%autoreload 2

import os
os.environ["CUDA_VISIBLE_DEVICES"]="1, 0"

In [ ]:
from main_coco import train

faster_rcnn, dataset = train()

loading annotations into memory...
Done (t=18.72s)
creating index...
index created!


Epoch 0:   0%|          | 0/7393 [00:00<?, ?it/s]/home/tim/.conda/envs/kaggle-auto/lib/python3.7/site-packages/torch/nn/parallel/_functions.py:61: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
Epoch 0:  22%|██▏       | 1608/7393 [32:45<1:57:36,  1.22s/it, loss=382, rpn_cls_loss=0.0593, rpn_reg_loss=10.5, rcnn_cls_loss=1.24, rcnn_reg_loss=27.6, accuracy=34.9, offsets=28.1]

In [ ]:
from torch.nn import functional as F
import torchvision.transforms as transforms
from torch import autograd, nn, optim, utils
from dataset_coco import detection_collate, CocoDataset


dataset = CocoDataset("data/coco/val2017", "data/coco/annotations/instances_val2017.json")
dataloader = utils.data.DataLoader(dataset, batch_size=1, num_workers=12, collate_fn=detection_collate)
dataloader_iter = iter(dataloader)

In [ ]:
img, targets, classes = next(dataloader_iter)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

import torch

img = img.float().cuda()
targets = targets.detach().float().cuda()
img_id = torch.arange(0, img.size()[0])

fig, ax = plt.subplots(figsize=(15, 15))
visualize_rpn(ax, targets.cpu()[0], classes.cpu()[0], img.cpu()[0])

In [ ]:
from lib.predictor_efficient import FasterRCNNPredictor
faster_rcnn_predictor = FasterRCNNPredictor(faster_rcnn).cuda()

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

import torch

img, targets, classes = next(dataloader_iter)
img = img.float().cuda()
targets = targets.detach().float().cuda()
img_id = torch.arange(0, img.size()[0])

nms_reg_rounded, nms_reg, nms_cls, rcnn_reg, rcnn_cls, reg, cls = faster_rcnn_predictor(img, img_id)

i = 1

fig, ax = plt.subplots(figsize=(10, 10))
np_img = img_to_np(img[i])
ax.imshow(np_img)
visualize_rpn(ax, nms_reg.cpu()[i], nms_cls.cpu()[i], img.cpu()[i], [1.0, 1.0, 1.0], draw_all=False)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
np_img = img_to_np(img[i])
ax.imshow(np_img)
visualize_rcnn(ax, reg[i].cpu(), cls[i].cpu(), color_map)
plt.show()

In [ ]:
rpn_reg, rpn_cls, nms_reg, nms_cls, rcnn_reg, rcnn_cls, anchors = faster_rcnn(img, img_id)
_, _, p_rcnn_reg, p_rcnn_cls, reg, cls = faster_rcnn_predictor(img, img_id)

In [ ]:
print(rcnn_cls.shape, rcnn_reg.shape) 

from torch.nn import functional as F

rcnn_cls_mask = F.softmax(rcnn_cls[0, :])[:, 0] < 0.1
p_rcnn_cls_mask = F.softmax(p_rcnn_cls[0, :])[:, 0] < 0.1

r = rcnn_reg[0][rcnn_cls_mask, :]
p_r = p_rcnn_reg[0][p_rcnn_cls_mask, :]
r_nms = nms_reg[0][rcnn_cls_mask, :]
r_nms_cls = nms_cls[0][rcnn_cls_mask]
print(r.shape, p_r.shape, r_nms.shape)

In [ ]:
r

In [ ]:
p_r

In [ ]:
r_nms

In [ ]:
F.softmax(rcnn_cls[0, rcnn_cls_mask, :])

In [ ]:
torch.sigmoid(r_nms_cls)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
visualize_rpn(ax, r_nms.cpu(), r_nms_cls.cpu(), img.cpu()[0], color_map, draw_all=True)

In [ ]:
nms_reg_rounded = torch.round(r_nms * faster_rcnn.reduction) // faster_rcnn.reduction

fig, ax = plt.subplots(figsize=(10, 10))
visualize_rpn(ax, nms_reg_rounded.cpu()[:2], r_nms_cls.cpu()[:2], img.cpu()[0], color_map, draw_all=True)

In [ ]:
rcnn_cls[0, :, 0]